In [ ]:
import datetime
import math
import cv2
import numpy as np

#global variables
width = 0
height = 0
entranceCounter = 0
exitCounter = 0
minContourArea = 600  #Adjust ths value according to tweak the size of the moving object found
binarizationThreshold = 30  #Adjust ths value to tweak the binarization
offsetEntranceLine = 30  #offset of the entrace line above the center of the image
offsetExitLine = 60 #offset of the entrance line below the center of the image

In [ ]:
#Check if an object in entering in monitored zone
def checkEntranceLineCrossing(y, coorYEntranceLine, coorYExitLine):
    absDistance = abs(y - coorYEntranceLine)

    if ((absDistance <= 2) and (y < coorYExitLine)):
        return 1
    else:
        return 0

In [ ]:
#Check if an object in exitting from monitored zone
def checkExitLineCrossing(y, CoorYEntranceLine, CoorYExitLine):
    AbsDistance = abs(y - CoorYExitLine)	

    if ((AbsDistance <= 2) and (y > CoorYEntranceLine)):
        return 1
    else:
        return 0

In [ ]:
def greyScaleConversion(frame):
    """
    Convert the image from 3 channels to greyscale to reduce the compute required to run it.
    """
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [ ]:
def gaussianBlurring(frame):
    """
    Preprocess the image by applying a gaussian blur
    """
    return cv2.GaussianBlur(frame, ksize =(21, 21), sigmaX = 0)

In [ ]:
def getImageDiff(referenceFrame, frame):
    """
    Get the difference between 2 frames to isolate and retrieve only the moving object
    """
    return cv2.absdiff(referenceFrame, frame)

In [ ]:
def thresholdImage(frame, binarizationThreshold=70):
    """
    Threshold the image to make it black and white. values above binarizationThreshold are made white and the rest
    is black
    """
    return cv2.threshold(frame, binarizationThreshold, 255, cv2.THRESH_BINARY)[1]

In [ ]:
def dilateImage(frame, interations=2 ):
    """
    Dilate the image to prevent spots that are black inside an image from being counted as individual objects
    """
    return cv2.dilate(frame, None, iterations=2)

In [ ]:
def getContours(frame):
    """
    Get the contours in the frame 
    @return: contours
    """
    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [ ]:
def getContourBound(contour):
    """
    Returns a rectangle that is a bound around the contour.
    @return: contour Bounds
    """
    (x,y,w,h) = cv2.boundingRect(contour)
    return (x,y,w,h)

In [ ]:
def getContourCentroid(x, y, w, h):
    """
    Get the centroid/center of the countours you have
    @return: The coordinates of the  center points
    """
    coordXCentroid = (x+x+w)/2
    coordYCentroid = (y+y+h)/2
    objectCentroid = (int(coordXCentroid),int(coordYCentroid))
    return objectCentroid

In [ ]:
camera = cv2.VideoCapture("Road traffic video for object recognition.mp4")

#force 640x480 webcam resolution
camera.set(3,640)
camera.set(4,480)

referenceFrame = None

#The webcam maybe get some time / captured frames to adapt to ambience lighting. For this reason, 
#some frames are grabbed and discarted.
for i in range(0,20):
    (grabbed, Frame) = camera.read()

while True:    
    (grabbed, Frame) = camera.read()
    height = np.size(Frame,0)
    width = np.size(Frame,1)

    #if cannot grab a frame, this program ends here.
    if not grabbed:
        break

    #gray-scale convertion and Gaussian blur filter applying
    grayFrame = greyScaleConversion(Frame)
    blurredFrame = gaussianBlurring(grayFrame)
    
    if referenceFrame is None:
        referenceFrame = blurredFrame
        continue

    #Background subtraction and image binarization
    frameDelta = getImageDiff(referenceFrame, blurredFrame)
    frameThresh = thresholdImage(frameDelta, binarizationThreshold)
    
    #Dilate image and find all the contours
    dilatedFrame = dilateImage(frameThresh)
    cnts = getContours(dilatedFrame)

    qttyOfContours = 0

    #plot reference lines (entrance and exit lines) 
    coordYEntranceLine = int((height / 2)-offsetEntranceLine)
    coordYExitLine = int((height / 2)+offsetExitLine)
    cv2.line(Frame, (0,coordYEntranceLine), (width,coordYEntranceLine), (255, 0, 0), 2)
    cv2.line(Frame, (0,coordYExitLine), (width,coordYExitLine), (0, 0, 255), 2)


    #check all found contours
    for c in cnts:
        #if a contour has small area, it'll be ignored
        if cv2.contourArea(c) < minContourArea:
            continue

        qttyOfContours = qttyOfContours+1    

        #draw an rectangle "around" the object
        (x, y, w, h) = getContourBound(c)
        cv2.rectangle(Frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        #find object's centroid
        objectCentroid = getContourCentroid(x, y, w, h)
        cv2.circle(Frame, objectCentroid, 1, (0, 0, 0), 5)
        
        coordXCentroid = (x+x+w)/2
        coordYCentroid = (y+y+h)/2
        
        if (checkEntranceLineCrossing(coordYCentroid,coordYEntranceLine,coordYExitLine)):
            entranceCounter += 1

        if (checkExitLineCrossing(coordYCentroid,coordYEntranceLine,coordYExitLine)):  
            exitCounter += 1

    #print("Total countours found: " +str(QttyOfContours))

    #Write entrance and exit counter values on frame and shows it
    cv2.putText(Frame, "Entrances: {}".format(str(entranceCounter)), (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (250, 0, 1), 2)
    cv2.putText(Frame, "Exits: {}".format(str(exitCounter)), (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imshow("Original Frame", Frame)
    cv2.waitKey(1);


# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

References
1. 